## 0. 전반적인 Process
- 현황
    - 샘플 데이터를 활용한 Prompt Engineering 수행 중
- 과제
    - Postgre DB 연계 필요
    - BPI 연계 필요

In [1]:
from IPython.display import Image, display
import base64

def mm(graph: str) -> None:
    """
    Mermaid 그래프를 이미지로 렌더링하여 표시합니다.

    이 함수는 Mermaid 문법으로 작성된 그래프 문자열을 받아
    base64로 인코딩한 후, Mermaid 온라인 렌더링 서비스를 통해
    이미지로 변환하여 Jupyter Notebook이나 유사한 환경에서 표시합니다.

    Args:
        graph (str): Mermaid 문법으로 작성된 그래프 문자열

    Returns:
        None: 이미지를 직접 표시하므로 반환값은 없습니다.

    Example:
        mm('''
        graph TD
        A[Start] --> B{Is it?}
        B -- Yes --> C[OK]
        B -- No --> D[End]
        ''')
    """
    # 그래프 문자열을 UTF-8로 인코딩
    graphbytes = graph.encode("utf-8")
    
    # 인코딩된 바이트를 base64로 변환
    base64_bytes = base64.b64encode(graphbytes)
    
    # base64 바이트를 ASCII 문자열로 디코딩
    base64_string = base64_bytes.decode("ascii")
    
    # Mermaid 온라인 렌더링 서비스 URL과 base64 문자열을 결합하여 이미지 표시
    display(Image(url="https://mermaid.ink/img/" + base64_string))

mm("""
graph LR
    A[API Key 등 협업 / 기획] --> B[DB 연계]
    B --> C[BPI 계산]
    C --> D[강약점 선택]
    D --> E[강약점 데이터전처리]
    E --> F[Prompt Engineering]
    F --> G[완료]
""")

## 1. 필수 라이브러리 임포트 및 환경 설정
- .env 파일에 '자신' 또는 '더존' api key 저장: 
    * OPENAI_API_KEY=sk-proj-.....

In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnableMap, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from typing import Dict, Any, Literal
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackHandler
from langchain.cache import SQLiteCache
from operator import itemgetter
from tabulate import tabulate
from langchain.callbacks import get_openai_callback
from langchain_teddynote import logging

# env 파일 읽기
load_dotenv()

# OpenAI API 키 가져오기
openai_api_key = os.getenv("OPENAI_API_KEY")

# 프로젝트 이름을 입력합니다.
logging.langsmith("AI경영진단보고서")

LangSmith 추적을 시작합니다.
[프로젝트명]
AI경영진단보고서


## 2. Sample Data 생성
- 데이터 출력 방식은 협의 필요 (\w 조희성 과장님, 이도은 대리님)
- 현재 출력물
    - df_company_info : 회사명, 업종, 재무/회계 데이터
    - df_partner_info : df_company_info의 회사의 매입/매출 거래처 정보 등
- 향후 : Postgre DB와 연계

In [3]:
import pandas as pd
import numpy as np

# 접근 시점 정의
ACCESS_TIME = '2024-09-01'
access_datetime = pd.to_datetime(ACCESS_TIME)

# 재현 가능성을 위해 난수 생성기의 시드를 설정합니다.
np.random.seed(42)

# 회사 및 업종 정보 생성
num_companies = 100  # 총 100개의 회사 생성
company_names = [f'Company_{i+1}' for i in range(num_companies)]  # 회사명 리스트 생성 (Company_1, Company_2, ..., Company_100)
industries = ['Manufacturing', 'Retail', 'Technology', 'Healthcare', 'Finance']  # 업종 리스트 정의

# 각 회사에 대한 고정된 업종 리스트 생성
company_industries = [np.random.choice(industries) for _ in range(num_companies)]  

# 날짜 생성 (2020-01 ~ ACCESS_TIME 기준 한 달 전까지)
dates = pd.date_range(start="2020-01-01", end=access_datetime - pd.DateOffset(months=1), freq='MS').strftime("%Y-%m").tolist()

# 상위 매출처 및 매입처 정보 생성 함수 정의
sales_partner_pool = list(set(f'SalesPartner_{i+1}' for i in range(8)))  # 매출처 Pool 생성 (200개)
purchase_partner_pool = list(set(f'PurchasePartner_{i+1}' for i in range(8)))  # 매입처 Pool 생성 (200개)1

# 각 회사별 기본 인원수를 설정합니다.
company_base_employees = {company: np.random.randint(100, 300) for company in company_names}  # 기업별로 100명 ~ 300명 사이의 기본 인원수 설정

def generate_partner_data():
    """
    상위 매출처 및 매입처 정보를 생성하는 함수입니다.
    
    Returns:
        dict: 생성된 매출처 및 매입처 정보 딕셔너리 리스트
    """
    # 상위 5개 매출처 및 매입처 무작위 선택
    sales_companies = list(np.random.choice(sales_partner_pool, 5, replace=False))
    sales_grades = list(np.random.choice(['AAA', 'AA+', 'AA', 'A+', 'A', 'BBB+', 'BBB', 'BB', 'B', 'CCC-'], 5, replace=False))

    purchase_companies = list(np.random.choice(purchase_partner_pool, 5, replace=False))
    purchase_grades = list(np.random.choice(['AAA', 'AA+', 'AA', 'A+', 'A', 'BBB+', 'BBB', 'BB', 'B', 'CCC-'], 5, replace=False))

    # 매출처 및 매입처 정보 딕셔너리 리스트로 생성
    sales_partners = [
        {
            "매출처_회사명": sales_companies[i],
            "매출처_신용등급": sales_grades[i],
            "매출처_매출금액": int(np.random.uniform(1000, 10000)),  # 월별 매출금액 (1000만원 ~ 10000만원)
            "매출처_회수금액": int(np.random.uniform(500, 5000)),  # 월별 회수금액 (100만원 ~ 1000만원)
            "매출처_회수기일": int(np.random.uniform(30, 90)),  # 월별 회수기일 (30일 ~ 90일)
        }
        for i in range(5)
    ]

    purchase_partners = [
        {
            "매입처_회사명": purchase_companies[i],
            "매입처_신용등급": purchase_grades[i],
            "매입처_매입금액": int(np.random.uniform(1000, 10000)),  # 월별 매입금액 (1000만원 ~ 10000만원)
            "매입처_지급금액": int(np.random.uniform(100, 1000)),  # 월별 지급금액 (100만원 ~ 1000만원)
            "매입처_지급기일": int(np.random.uniform(30, 90)),  # 월별 회수기일 (30일 ~ 90일)
        }
        for i in range(5)
    ]

    return sales_partners, purchase_partners

# 기본 정보 데이터 생성 함수 정의
def generate_basic_data(company, industry, date):
    """
    각 회사의 특정 날짜에 대한 기본 재무 데이터를 생성하는 함수입니다.
    
    Args:
        company (str): 회사명
        industry (str): 업종명
        date (str): 날짜 (YYYY-MM 형식)
    
    Returns:
        dict: 생성된 회사의 기본 재무 정보 및 상위 매출처, 매입처 정보
    """
    base_asset_value = 30000  # 기본 총자산 값 설정 (3억원)
    total_assets = round(base_asset_value * (1 + np.random.normal(0, 0.05)), 0)  # 기본 총자산 값에서 5% 변동 (정규분포 이용, 소수점 없이)
    revenue = round(np.random.uniform(1000, 10000), 0)  # 매출액은 1천만원 ~ 1억원 사이의 값으로 설정 (소수점 없이)
    operating_profit = round(revenue * np.random.uniform(-0.2, 0.2), 0)  # 영업이익은 매출액의 -20%~20% 범위로 설정 (소수점 없이)
    net_income = round(operating_profit * np.random.uniform(-0.5, 0.8), 0)  # 당기순이익은 영업이익의 -50%~80% 범위로 설정 (소수점 없이)
    short_term_loans = round(np.random.uniform(500, 5000), 0)  # 단기차입금은 500만원 ~ 5000만원 범위로 설정 (소수점 없이)
    long_term_loans = round(np.random.uniform(1000, 10000), 0)  # 장기차입금은 1000만원 ~ 1억원 범위로 설정 (소수점 없이)
    total_loans = short_term_loans + long_term_loans  # 총 차입금 계산
    loan_to_sales = round((total_loans / revenue) * 100, 2)  # 매출대비차입금 비율을 백분율로 계산
    working_capital_turnover = round(np.random.uniform(1, 5), 2)  # 운전자금회전율은 1~5회 사이의 값으로 설정
    operating_cash_flow = round(np.random.uniform(500, 5000), 0)  # 영업활동현금흐름은 500만원 ~ 5000만원 범위로 설정 (소수점 없이)
    net_cash_flow = round(operating_cash_flow - total_loans, 0)  # 순현금흐름 = 영업활동현금흐름 - 총 차입금 (소수점 없이)
    ar_balance = round(np.random.uniform(1000, 5000), 0)  # 매출채권 규모 (1000만원 ~ 5000만원)
    ap_balance = round(np.random.uniform(800, 4000), 0)  # 매입채무 규모 (800만원 ~ 4000만원)
    inventory = round(np.random.uniform(2000, 10000), 0)  # 재고자산 (2000만원 ~ 1억원)

    # 상위 매출처 및 매입처 정보 생성
    sales_partners, purchase_partners = generate_partner_data()

    # 기본 인원수를 기반으로 ±10% 내외의 변동을 줍니다.
    base_employees = company_base_employees[company]
    employees = round(base_employees * (1 + np.random.uniform(-0.1, 0.1)))  # ±10% 내외 변동

    return {
        '기업명': company,
        '업종': industry,
        '날짜': date,
        '매출액증가율': round(np.random.uniform(5, 15), 2),  # 매출액증가율 (5% ~ 15% 사이)
        '총자산증가율': round(np.random.uniform(3, 10), 2),  # 총자산증가율 (3% ~ 10% 사이)
        '총자산': int(total_assets),  # 총자산 (정수형)
        '매출액': int(revenue),  # 매출액 (정수형)
        '영업이익': int(operating_profit),  # 영업이익 (정수형)
        '영업이익률': round(operating_profit / revenue * 100, 2) if revenue != 0 else round(np.random.uniform(-20, 20), 2),  # 영업이익률 (백분율)
        '당기순이익': int(net_income),  # 당기순이익 (정수형)
        '당기순이익률': round(net_income / revenue * 100, 2) if revenue != 0 else round(np.random.uniform(-30, 30), 2),  # 당기순이익률 (백분율)
        '단기차입금': int(short_term_loans),  # 단기차입금 (정수형)
        '장기차입금': int(long_term_loans),  # 장기차입금 (정수형)
        '매출대비차입금': loan_to_sales,  # 매출대비차입금 비율 (백분율)
        '운전자금회전율': working_capital_turnover,  # 운전자금회전율
        '인원수': employees,  # 인원수 (±10% 변동)
        '월평균급여액': round(np.random.uniform(200, 500), 1),  # 월평균 급여액 (200 ~ 500만원)
        '월매출창출액': round(np.random.uniform(5000, 20000), 1),  # 월 매출 창출액 (5000 ~ 20000만원)
        '영업활동현금흐름/매출액': round(np.random.uniform(0.05, 0.2), 2),  # 영업활동현금흐름/매출액 비율 (5% ~ 20%)
        '영업활동현금흐름': int(operating_cash_flow),  # 영업활동현금흐름 (정수형)
        '순현금흐름': int(net_cash_flow),  # 순현금흐름 (정수형)
        '매출채권': int(ar_balance),  # 매출채권 (정수형)
        '매입채무': int(ap_balance),  # 매입채무 (정수형)
        '재고자산': int(inventory),  # 재고자산 (정수형)
        '상위_매출처': sales_partners,  # 상위 매출처 정보
        '상위_매입처': purchase_partners  # 상위 매입처 정보
    }

# 기본 정보 데이터 생성
# 각 회사와 날짜별로 기본 재무 데이터를 생성하여 리스트로 저장
basic_data = [
    generate_basic_data(company, industry, date)
    for company, industry in zip(company_names, company_industries)
    for date in dates
]

# 기본 정보 데이터프레임 생성
df_company_info = pd.DataFrame(basic_data)  # 회사 기본 정보로 데이터프레임 생성

# 데이터프레임 샘플 출력
print(df_company_info[['기업명', '날짜', '상위_매출처', '상위_매입처']].head())


         기업명       날짜                                             상위_매출처  \
0  Company_1  2020-01  [{'매출처_회사명': 'SalesPartner_3', '매출처_신용등급': 'CC...   
1  Company_1  2020-02  [{'매출처_회사명': 'SalesPartner_5', '매출처_신용등급': 'B'...   
2  Company_1  2020-03  [{'매출처_회사명': 'SalesPartner_4', '매출처_신용등급': 'BB...   
3  Company_1  2020-04  [{'매출처_회사명': 'SalesPartner_4', '매출처_신용등급': 'BB...   
4  Company_1  2020-05  [{'매출처_회사명': 'SalesPartner_5', '매출처_신용등급': 'AA...   

                                              상위_매입처  
0  [{'매입처_회사명': 'PurchasePartner_2', '매입처_신용등급': ...  
1  [{'매입처_회사명': 'PurchasePartner_7', '매입처_신용등급': ...  
2  [{'매입처_회사명': 'PurchasePartner_8', '매입처_신용등급': ...  
3  [{'매입처_회사명': 'PurchasePartner_1', '매입처_신용등급': ...  
4  [{'매입처_회사명': 'PurchasePartner_7', '매입처_신용등급': ...  


- Company_1에 대한 샘플 데이터

In [4]:
from pprint import pprint

print('#'*10, 'Company_1에 대한 정보', '#'*10)
pprint(df_company_info[df_company_info['기업명']== 'Company_1'])

df_company_info.to_csv("sample.csv", encoding='utf-8-sig', index=False)

########## Company_1에 대한 정보 ##########
          기업명          업종       날짜  매출액증가율  총자산증가율    총자산   매출액  영업이익  영업이익률  \
0   Company_1  Healthcare  2020-01   14.94    4.23  29011  3508   573  16.33   
1   Company_1  Healthcare  2020-02    5.71    5.78  30006  4298   420   9.77   
2   Company_1  Healthcare  2020-03   11.35    3.32  27659  7048  -484  -6.87   
3   Company_1  Healthcare  2020-04    8.55    9.70  33001  8708   553   6.35   
4   Company_1  Healthcare  2020-05   11.29    7.87  31685  2915   520  17.84   
5   Company_1  Healthcare  2020-06    5.47    4.88  33215  9110 -1656 -18.18   
6   Company_1  Healthcare  2020-07   11.88    6.58  29223  8347  1562  18.71   
7   Company_1  Healthcare  2020-08    5.05    9.83  31339  8815 -1465 -16.62   
8   Company_1  Healthcare  2020-09   10.32    3.47  30429  5290   716  13.53   
9   Company_1  Healthcare  2020-10    8.11    4.59  32675  3995   368   9.21   
10  Company_1  Healthcare  2020-11   11.72    3.14  30924  6433  -993 -15.44   
1

## 3. 데이터전처리
- 현황 : 성장성, 수익성 구현
- 전처리 함수
    - preprocess_growth_data : '총자산', '매출액' 등 성장성 관련 Data를 Dict 형태로 변경
    - preprocess_profitability_data : '영업이익', '당기순이익' 등 수익성 관련 Data를 Dict 형태로 변경

### 성장성 (Growth) 전처리

In [5]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Any
import logging

# 재현성을 위한 Random seed 설정
np.random.seed(42)

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 접근 시점 정의
ACCESS_TIME = '2024-09-01'
access_datetime = pd.to_datetime(ACCESS_TIME)

class DataValidationError(Exception):
    """데이터 검증 관련 커스텀 예외"""
    pass

class GrowthMetricsCalculator:
    """성장성 지표 계산을 위한 메인 클래스"""

    REQUIRED_COLUMNS = ['기업명', '날짜', '업종', '총자산', '매출액']
    ASSET_COL, REVENUE_COL = '총자산', '매출액'
    COMPANY_COL, DATE_COL, INDUSTRY_COL = '기업명', '날짜', '업종'

    def __init__(self, df_company_info: pd.DataFrame, target_company_name: str, n_years: int = 3):
        self.df = self._validate_input_data(df_company_info)
        self.target_company_name = target_company_name
        self.n_years = n_years
        self.growth_data = {
            'annual_revenue': {},
            'annual_assets': {},
            'monthly_revenue': {},
            'monthly_growth': {}
        }

    def _validate_input_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """필수 컬럼 및 데이터 유효성 검증 후 데이터 반환"""
        if not all(col in df.columns for col in self.REQUIRED_COLUMNS):
            missing_cols = set(self.REQUIRED_COLUMNS) - set(df.columns)
            raise DataValidationError(f"필수 컬럼 누락: {missing_cols}")
        if df[self.COMPANY_COL].isnull().all():
            raise DataValidationError(f"회사를 찾을 수 없음")
        for col in [self.ASSET_COL, self.REVENUE_COL]:
            if not pd.to_numeric(df[col], errors='coerce').notnull().all():
                raise DataValidationError(f"숫자가 아닌 값이 포함됨: {col}")
        logger.info("데이터 검증 완료")
        return df[self.REQUIRED_COLUMNS].copy()

    def _prepare_data(self):
        """데이터 전처리 및 필요한 필드 추가"""
        self.df['date'] = pd.to_datetime(self.df[self.DATE_COL])
        self.df['year'] = self.df['date'].dt.year
        self.df['month'] = self.df['date'].dt.month
        self.df = self.df.fillna({self.ASSET_COL: 0, self.REVENUE_COL: 0})
        self.df_company = self.df[self.df[self.COMPANY_COL] == self.target_company_name].reset_index(drop=True)
        if self.df_company.empty:
            raise DataValidationError(f"회사 데이터가 없음: {self.target_company_name}")
        self.target_industry = self.df_company[self.INDUSTRY_COL].iloc[0]
        self.df_industry = self.df[self.df[self.INDUSTRY_COL] == self.target_industry].reset_index(drop=True)
        self.latest_date = access_datetime - pd.DateOffset(months=1)
        logger.info(f"데이터 준비 완료 - 기준일자: {self.latest_date.strftime('%Y-%m-%d')}")

    def _calculate_growth_rate(self, current: float, previous: float, threshold: float = 0.0001) -> float:
        """성장률 계산"""
        if abs(previous) < threshold:
            logger.warning(f"이전 값이 임계값보다 작음 (previous: {previous})")
            return 0
        return round((current - previous) / previous * 100, 2)

    def _process_annual_data(self):
        """과거 연도 및 연말 예상 연도 데이터 계산 및 저장"""
        for year_offset in range(1, self.n_years + 1):
            target_year = self.latest_date.year - year_offset
            self._calculate_annual_metrics(target_year, estimate=False)
        self._calculate_annual_metrics(self.latest_date.year, estimate=True)  # 연말 예상 데이터

    def _calculate_annual_metrics(self, year: int, estimate: bool = False):
        """연간 또는 연말 예상 데이터 계산"""
        if estimate:
            # 2024년 9월까지 누적 매출액을 기반으로 연환산 수행
            current_data = self.df_company[(self.df_company['year'] == year) & (self.df_company['month'] <= self.latest_date.month)]
            current_revenue = int((current_data[self.REVENUE_COL].sum() / self.latest_date.month) * 12)
        else:
            # 기존 연도는 그대로 누적합 사용
            current_data = self.df_company[self.df_company['year'] == year]
            current_revenue = int(current_data[self.REVENUE_COL].sum())
        
        prev_data = self.df_company[self.df_company['year'] == year - 1]
        prev_revenue = int(prev_data[self.REVENUE_COL].sum() if not prev_data.empty else 0)

        # 자산 데이터 처리 (연환산 불필요)
        current_assets = int(current_data[self.ASSET_COL].iloc[-1] if not current_data.empty else 0)
        prev_assets = int(prev_data[self.ASSET_COL].iloc[-1] if not prev_data.empty else 0)

        # 산업 평균 계산
        current_industry_data = self.df_industry[self.df_industry['year'] == year]
        prev_industry_data = self.df_industry[self.df_industry['year'] == year - 1]

        industry_revenue = int(current_industry_data[self.REVENUE_COL].sum())
        prev_industry_revenue = int(prev_industry_data[self.REVENUE_COL].sum() if not prev_industry_data.empty else 0)
        industry_assets = int(current_industry_data[self.ASSET_COL].mean() if not current_industry_data.empty else 0)
        prev_industry_assets = int(prev_industry_data[self.ASSET_COL].mean() if not prev_industry_data.empty else 0)

        key = f"{year}년(E)" if estimate else f"{year}년"
        self.growth_data['annual_revenue'][key] = {
            '매출액': current_revenue,
            '매출액증가율': self._calculate_growth_rate(current_revenue, prev_revenue),
            '업종평균 매출액증가율': self._calculate_growth_rate(industry_revenue, prev_industry_revenue),
        }
        self.growth_data['annual_assets'][key] = {
            '총자산': current_assets,
            '총자산증가율': self._calculate_growth_rate(current_assets, prev_assets),
            '업종평균 총자산증가율': self._calculate_growth_rate(industry_assets, prev_industry_assets)
        }

    def _process_monthly_data(self):
        """최근 12개월 월별 데이터 계산 및 저장"""
        # 2023-08부터 시작하는 최근 12개월 생성
        past_12_months = pd.date_range(end=self.latest_date + pd.DateOffset(months=1), periods=12, freq='ME')
        cumulative_data = {'current': {'revenue': 0}, 'previous': {'revenue': 0}}

        for idx, date in enumerate(past_12_months):
            month_str = date.strftime('%Y-%m')
            current_month = self.df_company[self.df_company['date'].dt.strftime('%Y-%m') == month_str]
            prev_month = self.df_company[self.df_company['date'].dt.strftime('%Y-%m') == (date - pd.DateOffset(years=1)).strftime('%Y-%m')]

            current_revenue = int(current_month[self.REVENUE_COL].sum())
            prev_revenue = int(prev_month[self.REVENUE_COL].sum() if not prev_month.empty else 0)

            cumulative_data['current']['revenue'] += current_revenue
            cumulative_data['previous']['revenue'] += prev_revenue

            self.growth_data['monthly_revenue'][month_str] = {
                '당월매출액': current_revenue,
                '전년동월매출액': prev_revenue
            }
            self.growth_data['monthly_growth'][month_str] = {
                '매출액증가율': self._calculate_growth_rate(current_revenue, prev_revenue),
                '누적매출액증가율': self._calculate_growth_rate(
                    cumulative_data['current']['revenue'],
                    cumulative_data['previous']['revenue']
                )
            }
            logger.debug(f"{month_str} 월별 데이터 처리 완료")


    def calculate_growth_metrics(self) -> Dict[str, pd.DataFrame]:
        """성장성 지표 계산 실행"""
        self._prepare_data()
        self._process_annual_data()
        self._process_monthly_data()

        # DataFrames 생성 및 날짜 순 정렬 (Transpose를 한 번만 사용)
        annual_revenue_df = pd.DataFrame(self.growth_data['annual_revenue']).sort_index(axis=1)
        annual_assets_df = pd.DataFrame(self.growth_data['annual_assets']).sort_index(axis=1)
        monthly_revenue_df = pd.DataFrame(self.growth_data['monthly_revenue']).sort_index(axis=1)
        monthly_growth_df = pd.DataFrame(self.growth_data['monthly_growth']).sort_index(axis=1)

        return {
            'latest_year_month': self.latest_date.strftime('%Y-%m'),
            'annual_revenue': annual_revenue_df,
            'annual_assets': annual_assets_df,
            'monthly_revenue': monthly_revenue_df,
            'monthly_growth': monthly_growth_df
        }


def preprocess_growth_data(df_company_info: pd.DataFrame, target_company_name: str) -> Dict[str, Any]:
    """성장성 지표 데이터를 전처리하여 JSON 형식으로 변환"""
    calculator = GrowthMetricsCalculator(df_company_info, target_company_name)
    return calculator.calculate_growth_metrics()



growth_data = preprocess_growth_data(df_company_info, 'Company_1')
print('#'*10, 'last_year_month', '#'*10)
pprint(growth_data['latest_year_month'])
print()

print('#'*10, 'annual_revenue', '#'*10)
pprint(pd.DataFrame(growth_data['annual_revenue']))
print()

print('#'*10, 'annual_assets', '#'*10)
pprint(pd.DataFrame(growth_data['annual_assets']))
print()

print('#'*10, 'monthly_revenue', '#'*10)
pprint(pd.DataFrame(growth_data['monthly_revenue']))
print()

print('#'*10, 'monthly_growth', '#'*10)
pprint(pd.DataFrame(growth_data['monthly_growth']))

2024-11-18 09:45:35,908 - INFO - 데이터 검증 완료
2024-11-18 09:45:35,922 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01


########## last_year_month ##########
'2024-08'

########## annual_revenue ##########
                2021년     2022년     2023년  2024년(E)
매출액          51847.00  70339.00  54942.00  48169.00
매출액증가율         -31.18     35.67    -21.89    -12.33
업종평균 매출액증가율     -3.15     -3.95      6.01    -33.59

########## annual_assets ##########
                2021년     2022년     2023년  2024년(E)
총자산          27306.00  29499.00  28112.00  30136.00
총자산증가율          -6.02      8.03     -4.70      7.20
업종평균 총자산증가율      0.27     -0.39      0.08      0.22

########## monthly_revenue ##########
         2023-09  2023-10  2023-11  2023-12  2024-01  2024-02  2024-03  \
당월매출액       5482     3709     1948     9713     3426     7085     2127   
전년동월매출액     2765     8461     6426     5601     1656     6082     3405   

         2024-04  2024-05  2024-06  2024-07  2024-08  
당월매출액       1405     8072     5101     1745     3152  
전년동월매출액     5029     2067     3542     7544     4765  

########## monthly_growth #######

### Profitability (수익성) 전처리

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Any
import logging

# 재현성을 위한 Random seed 설정
np.random.seed(42)

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 접근 시점 정의
ACCESS_TIME = '2024-09-01'
access_datetime = pd.to_datetime(ACCESS_TIME)

class DataValidationError(Exception):
    """데이터 검증 관련 커스텀 예외"""
    pass

class ProfitabilityMetricsCalculator:
    """수익성 지표 계산을 위한 메인 클래스"""

    REQUIRED_COLUMNS = ['기업명', '날짜', '업종', '영업이익', '당기순이익', '매출액']
    OPERATING_PROFIT_COL, NET_PROFIT_COL, REVENUE_COL = '영업이익', '당기순이익', '매출액'
    COMPANY_COL, DATE_COL, INDUSTRY_COL = '기업명', '날짜', '업종'

    def __init__(self, df_company_info: pd.DataFrame, target_company_name: str, n_years: int = 3):
        self.df = self._validate_input_data(df_company_info)
        self.target_company_name = target_company_name
        self.n_years = n_years
        self.profitability_data = {
            'annual_profit': {},
            'annual_margins': {},
            'monthly_profit': {},
            'monthly_margins': {}
        }

    def _validate_input_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """필수 컬럼 및 데이터 유효성 검증 후 데이터 반환"""
        if not all(col in df.columns for col in self.REQUIRED_COLUMNS):
            missing_cols = set(self.REQUIRED_COLUMNS) - set(df.columns)
            raise DataValidationError(f"필수 컬럼 누락: {missing_cols}")
        if df[self.COMPANY_COL].isnull().all():
            raise DataValidationError(f"회사를 찾을 수 없음")
        for col in [self.OPERATING_PROFIT_COL, self.NET_PROFIT_COL, self.REVENUE_COL]:
            if not pd.to_numeric(df[col], errors='coerce').notnull().all():
                raise DataValidationError(f"숫자가 아닌 값이 포함됨: {col}")
        logger.info("데이터 검증 완료")
        return df[self.REQUIRED_COLUMNS].copy()

    def _prepare_data(self):
        """데이터 전처리 및 필요한 필드 추가"""
        self.df['date'] = pd.to_datetime(self.df[self.DATE_COL])
        self.df['year'] = self.df['date'].dt.year
        self.df['month'] = self.df['date'].dt.month
        self.df = self.df.fillna({self.OPERATING_PROFIT_COL: 0, self.NET_PROFIT_COL: 0, self.REVENUE_COL: 0})
        self.df_company = self.df[self.df[self.COMPANY_COL] == self.target_company_name].reset_index(drop=True)
        if self.df_company.empty:
            raise DataValidationError(f"회사 데이터가 없음: {self.target_company_name}")
        self.target_industry = self.df_company[self.INDUSTRY_COL].iloc[0]
        self.df_industry = self.df[self.df[self.INDUSTRY_COL] == self.target_industry].reset_index(drop=True)
        self.latest_date = access_datetime - pd.DateOffset(months=1)
        self.latest_month = self.latest_date.month
        logger.info(f"데이터 준비 완료 - 기준일자: {self.latest_date.strftime('%Y-%m-%d')}")

    def _calculate_margin_rate(self, profit: float, revenue: float, threshold: float = 0.0001) -> float:
        """이익률 계산"""
        if abs(revenue) < threshold:
            logger.warning(f"매출액이 임계값보다 작음 (revenue: {revenue})")
            return 0
        return round((profit / revenue * 100), 2)

    def _process_annual_data(self):
        """과거 연도 및 연말 예상 연도 데이터 계산 및 저장"""
        for year_offset in range(1, self.n_years + 1):
            target_year = self.latest_date.year - year_offset
            self._calculate_annual_metrics(target_year, estimate=False)
        self._calculate_annual_metrics(self.latest_date.year, estimate=True)  # 연말 예상 데이터

    def _calculate_annual_metrics(self, year: int, estimate: bool = False):
        """연간 또는 연말 예상 데이터 계산"""
        if estimate:
            current_data = self.df_company[(self.df_company['year'] == year) & (self.df_company['month'] <= self.latest_month)]
            annualized_metrics = self._calculate_annualized_metrics(current_data, is_industry=False)
            industry_metrics = self._calculate_annualized_metrics(self.df_industry[self.df_industry['year'] == year], is_industry=True)
        else:
            current_data = self.df_company[self.df_company['year'] == year]
            prev_data = self.df_company[self.df_company['year'] == year - 1]
            industry_data = self.df_industry[self.df_industry['year'] == year]
            annualized_metrics = self._calculate_metrics(current_data, prev_data)
            industry_metrics = self._calculate_metrics(industry_data, None)

        key = f"{year}년(E)" if estimate else f"{year}년"
        self.profitability_data['annual_profit'][key] = {
            '영업이익': annualized_metrics['operating_profit'],
            '당기순이익': annualized_metrics['net_profit']
        }
        self.profitability_data['annual_margins'][key] = {
            '영업이익률': annualized_metrics['operating_profit_margin'],
            '당기순이익률': annualized_metrics['net_profit_margin'],
            '업종평균 영업이익률': industry_metrics['operating_profit_margin'],
            '업종평균 당기순이익률': industry_metrics['net_profit_margin']
        }

    def _calculate_annualized_metrics(self, current_data: pd.DataFrame, is_industry: bool = False) -> Dict[str, float]:
        """연환산 지표 계산"""
        monthly_op = current_data[self.OPERATING_PROFIT_COL].sum()
        monthly_np = current_data[self.NET_PROFIT_COL].sum()
        monthly_rev = current_data[self.REVENUE_COL].sum()

        # 연환산 계산
        annualized_op = (monthly_op / self.latest_month) * 12
        annualized_np = (monthly_np / self.latest_month) * 12
        annualized_rev = (monthly_rev / self.latest_month) * 12

        return {
            'operating_profit': int(round(annualized_op)),
            'net_profit': int(round(annualized_np)),
            'operating_profit_margin': self._calculate_margin_rate(annualized_op, annualized_rev),
            'net_profit_margin': self._calculate_margin_rate(annualized_np, annualized_rev)
        }

    def _calculate_metrics(self, current_data: pd.DataFrame, prev_data: pd.DataFrame = None) -> Dict[str, float]:
        """연간 수익성 지표 계산"""
        current_op = current_data[self.OPERATING_PROFIT_COL].sum()
        current_np = current_data[self.NET_PROFIT_COL].sum()
        current_rev = current_data[self.REVENUE_COL].sum()

        return {
            'operating_profit': int(round(current_op)),
            'net_profit': int(round(current_np)),
            'operating_profit_margin': self._calculate_margin_rate(current_op, current_rev),
            'net_profit_margin': self._calculate_margin_rate(current_np, current_rev)
        }

    def _process_monthly_data(self):
        """최근 12개월 월별 데이터 계산 및 저장"""
        past_12_months = pd.date_range(end=self.latest_date + pd.DateOffset(months=1), periods=12, freq='ME')
        cumulative_data = {'current': {'operating_profit': 0, 'net_profit': 0, 'revenue': 0}}

        for idx, date in enumerate(past_12_months):
            month_str = date.strftime('%Y-%m')
            current_month = self.df_company[self.df_company['date'].dt.strftime('%Y-%m') == month_str]
            prev_month = self.df_company[self.df_company['date'].dt.strftime('%Y-%m') == (date - pd.DateOffset(years=1)).strftime('%Y-%m')]

            current_op = int(current_month[self.OPERATING_PROFIT_COL].sum())
            current_np = int(current_month[self.NET_PROFIT_COL].sum())
            current_rev = int(current_month[self.REVENUE_COL].sum())
            prev_op = int(prev_month[self.OPERATING_PROFIT_COL].sum()) if not prev_month.empty else 0
            prev_np = int(prev_month[self.NET_PROFIT_COL].sum()) if not prev_month.empty else 0

            cumulative_data['current']['operating_profit'] += current_op
            cumulative_data['current']['net_profit'] += current_np
            cumulative_data['current']['revenue'] += current_rev

            self.profitability_data['monthly_profit'][month_str] = {
                '당월영업이익': current_op,
                '전년동월영업이익': prev_op,
                '당월당기순이익': current_np,
                '전년동월당기순이익': prev_np
            }
            self.profitability_data['monthly_margins'][month_str] = {
                '영업이익률': self._calculate_margin_rate(current_op, current_rev),
                '누적영업이익률': self._calculate_margin_rate(cumulative_data['current']['operating_profit'], cumulative_data['current']['revenue']),
                '당기순이익률': self._calculate_margin_rate(current_np, current_rev),
                '누적순이익률': self._calculate_margin_rate(cumulative_data['current']['net_profit'], cumulative_data['current']['revenue'])
            }

    def calculate_profitability_metrics(self) -> Dict[str, pd.DataFrame]:
        """수익성 지표 계산 실행"""
        self._prepare_data()
        self._process_annual_data()
        self._process_monthly_data()

        # DataFrames 생성 및 날짜 순 정렬
        annual_profit_df = pd.DataFrame(self.profitability_data['annual_profit']).sort_index(axis=1)
        annual_margins_df = pd.DataFrame(self.profitability_data['annual_margins']).sort_index(axis=1)
        monthly_profit_df = pd.DataFrame(self.profitability_data['monthly_profit']).sort_index(axis=1)
        monthly_margins_df = pd.DataFrame(self.profitability_data['monthly_margins']).sort_index(axis=1)

        return {
            'latest_year_month': self.latest_date.strftime('%Y-%m'),
            'annual_profit': annual_profit_df,
            'annual_margins': annual_margins_df,
            'monthly_profit': monthly_profit_df,
            'monthly_margins': monthly_margins_df
        }

def preprocess_profitability_data(df_company_info: pd.DataFrame, target_company_name: str) -> Dict[str, Any]:
    """수익성 지표 데이터를 전처리하여 JSON 형식으로 변환"""
    calculator = ProfitabilityMetricsCalculator(df_company_info, target_company_name)
    return calculator.calculate_profitability_metrics()



profitability_data = preprocess_profitability_data(df_company_info, 'Company_1')

print('#'*10, 'last_year_month', '#'*10)
pprint(profitability_data['latest_year_month'])
print()

print('#'*10, 'annual_revenue', '#'*10)
pprint(pd.DataFrame(profitability_data['annual_profit']))
print()

print('#'*10, 'annual_assets', '#'*10)
pprint(pd.DataFrame(profitability_data['annual_margins']))
print()

print('#'*10, 'monthly_revenue', '#'*10)
pprint(pd.DataFrame(profitability_data['monthly_profit']))
print()

print('#'*10, 'monthly_growth', '#'*10)
pprint(pd.DataFrame(profitability_data['monthly_margins']))

2024-11-18 09:45:36,109 - INFO - 데이터 검증 완료
2024-11-18 09:45:36,116 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01


########## last_year_month ##########
'2024-08'

########## annual_revenue ##########
       2021년  2022년  2023년  2024년(E)
영업이익    3554   1075  -1115     -2126
당기순이익   1793   1516   -441      -856

########## annual_assets ##########
             2021년  2022년  2023년  2024년(E)
영업이익률         6.85   1.53  -2.03     -4.41
당기순이익률        3.46   2.16  -0.80     -1.78
업종평균 영업이익률   -0.07   0.63  -0.72      1.72
업종평균 당기순이익률   0.08  -0.01  -0.20      0.52

########## monthly_revenue ##########
           2023-09  2023-10  2023-11  2023-12  2024-01  2024-02  2024-03  \
당월영업이익         532        4     -216    -1684     -424    -1025     -255   
전년동월영업이익       123     1663      840      -17      -64     1047     -382   
당월당기순이익        -40       -2      -38     -577     -171     -259       56   
전년동월당기순이익      -17      941      655       -4        7      626      126   

           2024-04  2024-05  2024-06  2024-07  2024-08  
당월영업이익        -210      290      363     -172       16  
전년동월영업이익      -73

In [7]:
import pandas as pd
from typing import Dict, Any, List
from datetime import datetime
import os
import logging
from pprint import pprint

ACCESS_TIME = '2024-09-01'
ACCESS_DATE = pd.to_datetime(ACCESS_TIME)


class PartnerStabilityMetricsCalculator:
    """거래처 안정성 지표 계산 클래스"""
    
    REQUIRED_COLUMNS = ['기업명', '날짜', '상위_매출처', '상위_매입처']
    
    def __init__(self, df: pd.DataFrame, company: str):
        """
        Args:
            df: 회사 데이터가 포함된 DataFrame
            company: 대상 회사명
        """
        self.df = df.copy()
        self._validate_input_data()
        self.df.loc[:, '날짜'] = pd.to_datetime(self.df['날짜'])
        self.df_target = self.df[self.df['기업명'] == company]
        self.latest_date = (ACCESS_DATE - pd.DateOffset(months=1)).strftime('%Y-%m')
        
    def _validate_input_data(self) -> None:
        """입력 데이터 검증"""
        missing_cols = set(self.REQUIRED_COLUMNS) - set(self.df.columns)
        if missing_cols:
            raise ValueError(f"필수 컬럼 누락: {missing_cols}")

    def _process_partner_data(self, partner_type: str) -> Dict[str, Any]:
        """거래처 데이터 처리"""
        start_date = pd.to_datetime(self.latest_date) - pd.DateOffset(months=11)
        
        # 기간 필터링
        # period_data = self.df_target[
        #     (self.df_target['날짜'] >= start_date) & 
        #     (self.df_target['날짜'] <= pd.to_datetime(self.latest_date))
        # ]
        
        # period_data = self.df_target[
        # (self.df_target['날짜'].dt.strftime('%Y-%m') >= start_date.strftime('%Y-%m')) & 
        # (self.df_target['날짜'].dt.strftime('%Y-%m') <= self.latest_date)
        # ]
        
        period_data = self.df_target[
        (self.df_target['날짜'] >= start_date) & 
        (self.df_target['날짜'] <= pd.to_datetime(self.latest_date) + pd.offsets.MonthEnd(1))
        ]
        
        # 거래처 데이터 추출
        partner_key = f'상위_{partner_type}처'
        partner_data = []
        for _, row in period_data.iterrows():
            for partner in row[partner_key]:
                partner_dict = partner.copy()
                partner_dict['날짜'] = row['날짜']
                partner_data.append(partner_dict)
                
        if not partner_data:
            return {}

        # 컬럼명 설정
        company_col = f'{partner_type}처_회사명'
        grade_col = f'{partner_type}처_신용등급'
        amount_col = f'{partner_type}처_{partner_type}금액'
        term_col = f'{partner_type}처_회수기일' if partner_type == '매출' else f'{partner_type}처_지급기일'

        # 데이터프레임 생성
        partner_df = pd.DataFrame(partner_data)
        
        # Top 5 거래처 선정
        total_by_company = partner_df.groupby(company_col)[amount_col].sum()
        top_5_companies = total_by_company.nlargest(5).index.tolist()
        
        # 연간 데이터 생성
        annual_data = []
        total_amount = total_by_company.sum()
        top_5_amount = total_by_company[top_5_companies].sum()
        
        for company in top_5_companies:
            company_data = partner_df[partner_df[company_col] == company]
            annual_amount = total_by_company[company]
            
            annual_data.append({
                '회사명': company,
                '신용등급': company_data[grade_col].iloc[0],
                f'연간_{partner_type}금액': annual_amount,
                '거래비율': round((annual_amount / total_amount * 100), 2)
            })
        
        # 합계 정보 추가
        annual_data.extend([
            {
                '회사명': '상위 5개 거래처 합계',
                f'연간_{partner_type}금액': top_5_amount,
                '거래비율': round((top_5_amount / total_amount * 100), 2)
            },
            {
                '회사명': '기타 거래처 합계',
                f'연간_{partner_type}금액': total_amount - top_5_amount,
                '거래비율': round(((total_amount - top_5_amount) / total_amount * 100), 2)
            }
        ])
        
        # 월별 데이터 생성
        monthly_data = {}
        for company in top_5_companies:
            company_data = partner_df[partner_df[company_col] == company]
            company_monthly = []
            
            # 전체 기간의 합계 계산
            total_term = company_data[term_col].sum()
            annual_amount = total_by_company[company]
            
            # 최근 12개월 데이터 추가
            # for month in pd.date_range(start=start_date, end=pd.to_datetime(self.latest_date), freq='ME'):
            #     month_data = company_data[
            #         company_data['날짜'].dt.to_period('M') == month.to_period('M')
            #     ]
            
            for month in pd.date_range(start=start_date, end=pd.to_datetime(self.latest_date) + pd.offsets.MonthEnd(1), freq='ME'):
                month_data = company_data[
                company_data['날짜'].dt.to_period('M') == month.to_period('M')
                ]
                
                if not month_data.empty:
                    amount = month_data[amount_col].sum()
                    company_monthly.append({
                        '날짜': month.strftime('%Y-%m'),
                        f'{partner_type}금액': amount
                    })

            # 평균 데이터 추가 (extend 사용)
            company_monthly.extend([
                {
                    '날짜': f'평균{partner_type}금액',
                    f'{partner_type}금액': round(annual_amount / 12, 2)
                },
                {
                    '날짜': f'평균{"회수" if partner_type=="매출" else "지급"}기일',
                    f'{partner_type}금액': round(total_term / 12, 2)
                }
            ])
            
            monthly_data[company] = company_monthly

        return {
            'annual_data': annual_data,
            'monthly_data': monthly_data
        }

    def calculate_stability_metrics(self) -> Dict[str, Any]:
        """안정성 지표 계산 실행"""
        sales_data = self._process_partner_data('매출')
        purchase_data = self._process_partner_data('매입')
        
        return {
            'latest_year_month': self.latest_date,
            'annual_top5_sales': sales_data.get('annual_data', []),
            'monthly_top5_sales': sales_data.get('monthly_data', {}),
            'annual_top5_purchase': purchase_data.get('annual_data', []),
            'monthly_top5_purchase': purchase_data.get('monthly_data', {})
        }

def preprocess_partner_stability_data(df: pd.DataFrame, company: str) -> Dict[str, Any]:
    """안정성 지표 전처리 함수"""
    calculator = PartnerStabilityMetricsCalculator(df, company)
    return calculator.calculate_stability_metrics()

# 'Company_1'에 대한 안정성 지표를 계산
company_1_stability = preprocess_partner_stability_data(df_company_info, 'Company_1')
pprint(company_1_stability)


{'annual_top5_purchase': [{'거래비율': 17.09,
                           '신용등급': 'AAA',
                           '연간_매입금액': 54284,
                           '회사명': 'PurchasePartner_7'},
                          {'거래비율': 14.62,
                           '신용등급': 'A+',
                           '연간_매입금액': 46436,
                           '회사명': 'PurchasePartner_1'},
                          {'거래비율': 13.56,
                           '신용등급': 'A+',
                           '연간_매입금액': 43061,
                           '회사명': 'PurchasePartner_4'},
                          {'거래비율': 12.31,
                           '신용등급': 'AA+',
                           '연간_매입금액': 39093,
                           '회사명': 'PurchasePartner_2'},
                          {'거래비율': 12.19,
                           '신용등급': 'BBB',
                           '연간_매입금액': 38721,
                           '회사명': 'PurchasePartner_3'},
                          {'거래비율': 69.78,
                           '연간_매입금액

## 4. Langchain을 활용한 분석

- cache 설정 : 비용 절감 목적
- streaming을 위한 custom handler 설정
- common_llm : 데이터 기반 해석만 하는 자유도가 낮은 gpt-4o-mini
- load_template : template 읽기
- determine_strength_weakness : **BPI 계산법 통해 연계 필요**
- create_analysis_chain : 분석 chain 생성
- run_analysis : 강/약점을 활용해 분석 시작

In [15]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, Any
import logging
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from operator import itemgetter
from tabulate import tabulate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import random
from tabulate import tabulate

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# OpenAI GPT-4 Mini 모델 설정
common_llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=300,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# 디렉토리 설정
CURRENT_DIR = os.getcwd()
PROMPT_PATH = os.path.join(CURRENT_DIR, "prompts")

def load_prompt(file_name: str) -> str:
    file_path = os.path.join(PROMPT_PATH, file_name)
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: Prompt file {file_name} not found.")
        return ""

def determine_strength_weakness(data: pd.DataFrame, company_name: str) -> Dict[str, str]:
    # logging.disable(logging.CRITICAL)  # 로깅 임시 비활성화
    growth_data = preprocess_growth_data(data[data['기업명'] == company_name], company_name)
    profitability_data = preprocess_profitability_data(data[data['기업명'] == company_name], company_name)
    partner_stability_data = preprocess_partner_stability_data(data[data['기업명'] == company_name], company_name)
    
    # 임의로 strength와 weakness를 선택하도록 변경
    indicators = ['growth', 'profitability', 'partner_stability']
    strength = random.choice(indicators)
    
    # weakness는 strength와 다른 값이 선택되도록 설정
    remaining_indicators = [indicator for indicator in indicators if indicator != strength]
    weakness = random.choice(remaining_indicators)
    
    return {'strength': strength, 'weakness': weakness}

def create_analysis_chain(indicator: str, is_strength: bool, llm: ChatOpenAI, df: pd.DataFrame, company_name: str):
    base_template = load_prompt(f"{indicator}_template_original.txt")
    additional_prompt = "Perform analysis on positive side." if is_strength else "Perform analysis on negative side."
    full_template = f"{base_template}\n\n{additional_prompt}"
    
    prompt = PromptTemplate.from_template(full_template)
    analysis_chain = prompt | llm | StrOutputParser()
    
    if indicator == 'growth':
        preprocess_func = preprocess_growth_data
        data_keys = ['latest_year_month', 'annual_revenue', 'annual_assets', 'monthly_revenue', 'monthly_growth']
    elif indicator == 'profitability':
        preprocess_func = preprocess_profitability_data
        data_keys = ['latest_year_month', 'annual_profit', 'annual_margins', 'monthly_profit', 'monthly_margins']
    elif indicator == 'partner_stability':
        preprocess_func = preprocess_partner_stability_data
        data_keys = [
            'latest_year_month',
            'annual_top5_sales',
            'monthly_top5_sales',
            'annual_top5_purchase',
            'monthly_top5_purchase'
        ]
    else:
        raise ValueError(f"Unknown indicator: {indicator}")
        
    return RunnableLambda(lambda df: analysis_chain.invoke({
        **{key: preprocess_func(df, company_name)[key] for key in data_keys}
    }))

def run_analysis(df: pd.DataFrame, company_name: str) -> Dict[str, Any]:
    strength_weakness = determine_strength_weakness(df, company_name)
    strength = strength_weakness['strength']
    weakness = strength_weakness['weakness']
    
    strength_chain = create_analysis_chain(strength, True, common_llm, df, company_name)
    weakness_chain = create_analysis_chain(weakness, False, common_llm, df, company_name)
    
    parallel_chain = RunnableParallel({
        'strength_analysis': strength_chain,
        'weakness_analysis': weakness_chain
    })
    
    return {
        'parallel': parallel_chain,
        'strength': strength_chain,
        'weakness': weakness_chain,
        'strength_name': strength,
        'weakness_name': weakness
    }


def print_preprocessed_data(df: pd.DataFrame, company_name: str, metric: str) -> None:
    """전처리된 데이터를 출력"""
    preprocess_func = globals()[f"preprocess_{metric}_data"]
    preprocessed_data = preprocess_func(df, company_name)
    latest_year_month = preprocessed_data['latest_year_month']
    print(f"\n{metric.capitalize()} Data ({latest_year_month} 기준):")
    
    if metric == 'growth':
        # Annual Revenue
        print("\nAnnual Revenue:")
        annual_revenue_df = pd.DataFrame(preprocessed_data['annual_revenue'])
        print(tabulate(annual_revenue_df, headers='keys', tablefmt='pretty'))

        # Annual Assets
        print("\nAnnual Assets:")
        annual_assets_df = pd.DataFrame(preprocessed_data['annual_assets'])
        print(tabulate(annual_assets_df, headers='keys', tablefmt='pretty'))

        # Monthly Revenue
        print("\nMonthly Revenue:")
        monthly_revenue_df = pd.DataFrame(preprocessed_data['monthly_revenue'])
        print(tabulate(monthly_revenue_df, headers='keys', tablefmt='pretty'))

        # Monthly Growth
        print("\nMonthly Growth:")
        monthly_growth_df = pd.DataFrame(preprocessed_data['monthly_growth'])
        print(tabulate(monthly_growth_df, headers='keys', tablefmt='pretty'))

    elif metric == 'profitability':
        # Annual Profit
        print("\nAnnual Profit:")
        annual_profit_df = pd.DataFrame(preprocessed_data['annual_profit'])
        print(tabulate(annual_profit_df, headers='keys', tablefmt='pretty'))

        # Annual Margins
        print("\nAnnual Margins:")
        annual_margins_df = pd.DataFrame(preprocessed_data['annual_margins'])
        print(tabulate(annual_margins_df, headers='keys', tablefmt='pretty'))

        # Monthly Profit
        print("\nMonthly Profit:")
        monthly_profit_df = pd.DataFrame(preprocessed_data['monthly_profit'])
        print(tabulate(monthly_profit_df, headers='keys', tablefmt='pretty'))

        # Monthly Margins
        print("\nMonthly Margins:")
        monthly_margins_df = pd.DataFrame(preprocessed_data['monthly_margins'])
        print(tabulate(monthly_margins_df, headers='keys', tablefmt='pretty'))

    elif metric == 'partner_stability':
        # Annual Top 5 Sales Partners
        print("\nAnnual Top 5 Sales Partners:")
        sales_df = pd.DataFrame(preprocessed_data['annual_top5_sales'])
        sales_df = sales_df.fillna('-')
        print(tabulate(sales_df, headers='keys', tablefmt='pretty', showindex=False))
        
        # Monthly Sales Data
        print("\nMonthly Sales Data (Top 5 Partners):")
        monthly_sales_combined_data = []
        
        for partner, transactions in preprocessed_data['monthly_top5_sales'].items():
            for transaction in transactions:
                transaction['회사명'] = partner
                # 매출금액을 정수로 변환 (평균 관련 컬럼 제외)
                if '평균' not in transaction['날짜'] and '매출금액' in transaction:
                    transaction['매출금액'] = int(transaction['매출금액'])
                monthly_sales_combined_data.append(transaction)
        
        monthly_sales_df = pd.DataFrame(monthly_sales_combined_data)
        
        # 피벗 테이블 생성
        monthly_sales_pivot = monthly_sales_df.pivot_table(
            index='회사명',
            columns='날짜',
            values='매출금액',
            aggfunc='first'
        ).reset_index()
        
        # Annual Top 5 순서대로 정렬하기 위한 순서 리스트 생성
        annual_order = [partner['회사명'] for partner in preprocessed_data['annual_top5_sales'] 
                       if partner['회사명'] not in ['상위 5개 거래처 합계', '기타 거래처 합계']]
        
        # 해당 순서로 정렬
        monthly_sales_pivot['sort_order'] = monthly_sales_pivot['회사명'].map(
            {company: idx for idx, company in enumerate(annual_order)}
        )
        monthly_sales_pivot = monthly_sales_pivot.sort_values('sort_order').drop('sort_order', axis=1)
        
        # 평균값 데이터 준비
        avg_sales = monthly_sales_df[
            monthly_sales_df['날짜'].str.startswith('평균')
        ].copy()
        
        # 평균값 컬럼 추가
        avg_sales_data = {}
        for partner in avg_sales['회사명'].unique():
            partner_data = avg_sales[avg_sales['회사명'] == partner]
            avg_sales_data[partner] = {
                '평균매출금액': partner_data[partner_data['날짜'] == '평균매출금액']['매출금액'].iloc[0],
                '평균회수기일': partner_data[partner_data['날짜'] == '평균회수기일']['매출금액'].iloc[0]
            }
        
        # latest_year_month 추가 및 컬럼 순서 지정
        all_months = sorted([col for col in monthly_sales_pivot.columns if col not in ['회사명', '평균매출금액', '평균회수기일']])
        latest_month = pd.to_datetime(preprocessed_data['latest_year_month']).strftime('%Y-%m')
        
        # 해당 월의 데이터가 있는지 확인
        if latest_month not in all_months:
            monthly_sales_pivot[latest_month] = '-'
            all_months.append(latest_month)
        
        # 컬럼 순서 지정
        monthly_sales_pivot = monthly_sales_pivot[['회사명'] + sorted(all_months)]
        
        # 평균값 컬럼 추가
        monthly_sales_pivot['평균매출금액'] = monthly_sales_pivot['회사명'].map(
            lambda x: avg_sales_data[x]['평균매출금액'] if x in avg_sales_data else '-'
        )
        monthly_sales_pivot['평균회수기일'] = monthly_sales_pivot['회사명'].map(
            lambda x: avg_sales_data[x]['평균회수기일'] if x in avg_sales_data else '-'
        )
        
        # NaN 값을 '-'로 대체
        monthly_sales_pivot = monthly_sales_pivot.fillna('-')
        print(tabulate(monthly_sales_pivot, headers='keys', tablefmt='pretty', showindex=False))

        # Annual Top 5 Purchase Partners
        print("\nAnnual Top 5 Purchase Partners:")
        purchase_df = pd.DataFrame(preprocessed_data['annual_top5_purchase'])
        purchase_df = purchase_df.fillna('-')
        print(tabulate(purchase_df, headers='keys', tablefmt='pretty', showindex=False))
        
        # Monthly Purchase Data (동일한 로직 적용)
        print("\nMonthly Purchase Data (Top 5 Partners):")
        monthly_purchase_combined_data = []
        
        for partner, transactions in preprocessed_data['monthly_top5_purchase'].items():
            for transaction in transactions:
                transaction['회사명'] = partner
                if '평균' not in transaction['날짜'] and '매입금액' in transaction:
                    transaction['매입금액'] = int(transaction['매입금액'])
                monthly_purchase_combined_data.append(transaction)
        
        monthly_purchase_df = pd.DataFrame(monthly_purchase_combined_data)
        
        monthly_purchase_pivot = monthly_purchase_df.pivot_table(
            index='회사명',
            columns='날짜',
            values='매입금액',
            aggfunc='first'
        ).reset_index()
        
        annual_order = [partner['회사명'] for partner in preprocessed_data['annual_top5_purchase'] 
                       if partner['회사명'] not in ['상위 5개 거래처 합계', '기타 거래처 합계']]
        
        monthly_purchase_pivot['sort_order'] = monthly_purchase_pivot['회사명'].map(
            {company: idx for idx, company in enumerate(annual_order)}
        )
        monthly_purchase_pivot = monthly_purchase_pivot.sort_values('sort_order').drop('sort_order', axis=1)
        
        avg_purchase = monthly_purchase_df[
            monthly_purchase_df['날짜'].str.startswith('평균')
        ].copy()
        
        avg_purchase_data = {}
        for partner in avg_purchase['회사명'].unique():
            partner_data = avg_purchase[avg_purchase['회사명'] == partner]
            avg_purchase_data[partner] = {
                '평균매입금액': partner_data[partner_data['날짜'] == '평균매입금액']['매입금액'].iloc[0],
                '평균지급기일': partner_data[partner_data['날짜'] == '평균지급기일']['매입금액'].iloc[0]
            }
        
        all_months = sorted([col for col in monthly_purchase_pivot.columns if col not in ['회사명', '평균매입금액', '평균지급기일']])
        if latest_month not in all_months:
            monthly_purchase_pivot[latest_month] = '-'
            all_months.append(latest_month)
        
        monthly_purchase_pivot = monthly_purchase_pivot[['회사명'] + sorted(all_months)]
        
        monthly_purchase_pivot['평균매입금액'] = monthly_purchase_pivot['회사명'].map(
            lambda x: avg_purchase_data[x]['평균매입금액'] if x in avg_purchase_data else '-'
        )
        monthly_purchase_pivot['평균지급기일'] = monthly_purchase_pivot['회사명'].map(
            lambda x: avg_purchase_data[x]['평균지급기일'] if x in avg_purchase_data else '-'
        )
        
        monthly_purchase_pivot = monthly_purchase_pivot.fillna('-')
        print(tabulate(monthly_purchase_pivot, headers='keys', tablefmt='pretty', showindex=False))

    else:
        print(f"Unknown metric: {metric}")





def merge_analysis_results(strength_result: str, weakness_result: str) -> str:
    """
    강점 분석 결과와 약점 분석 결과를 하나의 문자열로 병합합니다.
    """
    return '\n\n'.join([
        "Strength Analysis:",
        strength_result,
        "Weakness Analysis:",
        weakness_result
    ])

## 5. 경영제언 Session
- gpt-4o로 고품질 모델 활용
- 창의도를 결정하는 temperature=1.2로 설정
- max_tokens=500으로 조금 더 긴 text 작성하도록 설정
- final_chain : 앞에서 분석한 strength와 weakness를 합쳐서 경영제언을 하도록 chain으로 연결
    - 1안 : 앞에서 streaming으로 분석한 strength_result와 weakness_result를 묶어서 수행 (토큰 비용 아낌)
    - 2안 : End-to-End로 강단점 분석부터 시작해서 경영 제언까지 수행 (토큰 비용 더 발생) 

In [16]:
insight_llm = ChatOpenAI(
    model_name="gpt-4o",
    temperature=1.2,
    max_tokens=500,
    streaming=True,
    # cache=langchain_cache,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [17]:
# 분석 실행
company_list = df_company_info['기업명'].unique()[:1]
for firm in company_list:
    firm_data = df_company_info[df_company_info['기업명'] == firm]
    chains = run_analysis(df_company_info, firm)
    
    strength_chain = chains['strength']
    weakness_chain = chains['weakness']
    strength_name = chains['strength_name']
    weakness_name = chains['weakness_name']
    
    print(f"\n\n{'#'*10} Preprocessing Strength ({strength_name}) Data for {firm} {'#'*10}")
    print_preprocessed_data(df_company_info, firm, strength_name)
    
    print(f"\n\n{'#'*10} Analyzing Strength ({strength_name}) for {firm} {'#'*10}")
    strength_result = strength_chain.invoke(firm_data)
    # print(strength_result)
    
    print(f"\n\n{'#'*10} Preprocessing Weakness ({weakness_name}) Data for {firm} {'#'*10}")
    print_preprocessed_data(df_company_info, firm, weakness_name)
        
    print(f"\n\n{'#'*10} Analyzing Weakness ({weakness_name}) for {firm} {'#'*10}")
    weakness_result = weakness_chain.invoke(firm_data)
    # print(weakness_result)
    
    print("\n\n",'#'*10, f'Final Comment for {firm}', '#'*10)    
    final_chain = (
        RunnableParallel({
            'strength_result': itemgetter('strength'),
            'weakness_result': itemgetter('weakness')
        })
        | RunnableLambda(lambda x: {'info': merge_analysis_results(x['strength_result'], x['weakness_result'])})
        | PromptTemplate.from_template(
        """
        Translate and analyze the provided company's strengths and weaknesses:

        {info}

        - Analyze the company's strengths and weaknesses based on the provided data, focusing on key business management insights.
        - Provide specific and actionable recommendations, grounded in the analysis, to capitalize on strengths and address weaknesses.
        - Highlight critical trends and figures as evidence to support your recommendations.
        - Ensure that the recommendations are concise (limited to 5 lines) and translated into Korean in a professional tone.

        # Output Format
        - The response should be in Korean, using natural and connected language.
        - Limit the response to 5 lines or less, with a clear and cohesive structure.
        - Write in a continuous paragraph format that flows smoothly between data points, avoiding lists or bullet points.

        # Guidelines
        1. Base your response objectively on the given data without speculation.
        2. Use specific figures and trends to support your analysis.
        3. Provide practical recommendations that are directly actionable.
        4. Maintain clarity and professionalism in both the analysis and the translation to Korean.
        """
        ) 
        | insight_llm | StrOutputParser()
    )
    result = final_chain.invoke({
        'strength': strength_result,
        'weakness': weakness_result
    })
    
    # print(result)


2024-11-18 09:48:19,937 - INFO - 데이터 검증 완료
2024-11-18 09:48:19,941 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-11-18 09:48:19,961 - INFO - 데이터 검증 완료
2024-11-18 09:48:19,968 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01




########## Preprocessing Strength (partner_stability) Data for Company_1 ##########

Partner_stability Data (2024-08 기준):

Annual Top 5 Sales Partners:
+----------------------+----------+---------------+----------+
|        회사명        | 신용등급 | 연간_매출금액 | 거래비율 |
+----------------------+----------+---------------+----------+
|    SalesPartner_7    |    B     |     55190     |  16.44   |
|    SalesPartner_4    |   CCC-   |     52241     |  15.56   |
|    SalesPartner_1    |    BB    |     49265     |  14.67   |
|    SalesPartner_3    |    BB    |     42279     |  12.59   |
|    SalesPartner_2    |    B     |     40853     |  12.17   |
| 상위 5개 거래처 합계 |    -     |    239828     |  71.44   |
|   기타 거래처 합계   |    -     |     95892     |  28.56   |
+----------------------+----------+---------------+----------+

Monthly Sales Data (Top 5 Partners):
+----------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------

2024-11-18 09:48:20,969 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


상위 5개 매출처의 연간 매출금액 합계는 239,828백만원으로 전체 매출의 71.44%를 차지하며, 이 중 신용등급이 'B'와 'BB'인 거래처가 다수를 차지하고 있어 신용 리스크가 존재합니다. 특히, SalesPartner_4는 신용등급이 'CCC-'로 낮아 2023년 11월부터 신용도 저하가 우려되며, 이로 인해 거래 안정성이 감소할 수 있습니다. 매출처의 평균 회수 기일은 48일로 안정적이나, SalesPartner_7의 경우 2024년 7월에 평균 매출이 급감하여 2,240백만원에 그쳤습니다. 반면, 상위 5개 매입처는 연간 매입금액이 221,595백만원으로 전체 매입의 69.78%를 차지하며, 신용등급이 'AAA'와 'A+'인 거래처가 포함되어 있어 안정적인 공급망을 유지하고 있습니다.

2024-11-18 09:48:24,180 - INFO - 데이터 검증 완료
2024-11-18 09:48:24,191 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-11-18 09:48:24,222 - INFO - 데이터 검증 완료




########## Preprocessing Weakness (profitability) Data for Company_1 ##########

Profitability Data (2024-08 기준):

Annual Profit:
+------------+--------+--------+--------+-----------+
|            | 2021년 | 2022년 | 2023년 | 2024년(E) |
+------------+--------+--------+--------+-----------+
|  영업이익  |  3554  |  1075  | -1115  |   -2126   |
| 당기순이익 |  1793  |  1516  |  -441  |   -856    |
+------------+--------+--------+--------+-----------+

Annual Margins:
+-----------------------+--------+--------+--------+-----------+
|                       | 2021년 | 2022년 | 2023년 | 2024년(E) |
+-----------------------+--------+--------+--------+-----------+
|      영업이익률       |  6.85  |  1.53  | -2.03  |   -4.41   |
|     당기순이익률      |  3.46  |  2.16  |  -0.8  |   -1.78   |
|  업종평균 영업이익률  | -0.07  |  0.63  | -0.72  |   1.72    |
| 업종평균 당기순이익률 |  0.08  | -0.01  |  -0.2  |   0.52    |
+-----------------------+--------+--------+--------+-----------+

Monthly Profit:
+--------------------+---------+-----

2024-11-18 09:48:24,226 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-11-18 09:48:24,245 - INFO - 데이터 검증 완료
2024-11-18 09:48:24,248 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-11-18 09:48:24,272 - INFO - 데이터 검증 완료
2024-11-18 09:48:24,277 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-11-18 09:48:24,300 - INFO - 데이터 검증 완료
2024-11-18 09:48:24,304 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-11-18 09:48:24,322 - INFO - 데이터 검증 완료
2024-11-18 09:48:24,328 - INFO - 데이터 준비 완료 - 기준일자: 2024-08-01
2024-11-18 09:48:24,900 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2021년부터 2024년까지의 영업이익과 당기순이익은 각각 3,554백만 원에서 -2,126백만 원, 1,793백만 원에서 -856백만 원으로 급격히 감소하였으며, 이는 지속적인 적자를 나타냅니다. 2024년의 영업이익률과 당기순이익률은 각각 -4.41%와 -1.78%로, 업종 평균에 비해 현저히 낮아져 기업의 수익성이 심각하게 악화되었음을 보여줍니다. 월별 데이터에서도 2024년 1월부터 8월까지 영업이익이 -1,684백만 원에서 16백만 원으로 변동하며, 전년 동월 대비 대부분의 기간에서 부정적인 실적을 기록하여 회복의 기미가 보이지 않습니다.

 ########## Final Comment for Company_1 ##########


2024-11-18 09:48:30,066 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


상위 5개 매출처에 대한 의존도가 높아 신용 리스크가 있으며, 특히 신용등급이 낮은 SalesPartner_4와의 거래 안정성이 우려됩니다. 반면, 상위 매입처는 신용등급이 'AAA' 및 'A+'라 안정적인 공급망이 강점으로 작용합니다. 하지만 2021년부터 2024년까지 영업이익과 당기순이익이 각각 -2,126백만 원과 -856백만 원으로 급감하고, 영업이익률과 당기순이익률이 업종 평균을 크게 하회하여 수익성 악화가 지속되고 있습니다. 따라서 매출 다각화를 통해 신용 리스크를 완화하고, 비용 절감을 통한 경영 효율화로 손실 구조를 개선하는 것이 필요합니다. 안정적인 매입처와의 관계를 보강해 공급망 안정성을 유지함으로써 장기적으로 수익성을 회복할 수 있을 것입니다.

In [11]:
# pprint(final_chain)
print(result['strength'])

TypeError: string indices must be integers, not 'str'

In [ ]:


print(tabulate(pd.DataFrame(preprocess_growth_data(firm_data)['year_level_data']), headers = 'keys', tablefmt='psql'))

# pd.DataFrame(preprocess_growth_data(firm_data)['year_level_data'])

In [ ]:
# run_analysis 함수의 결과를 사용하여 final_chain 구성
def create_final_chain(df: pd.DataFrame):
    chains = run_analysis(df)
    parallel_chain = chains['parallel']

    final_chain = (
        parallel_chain
        | RunnableLambda(lambda x: {
            'info': merge_analysis_results(
                x['strength_analysis'],
                x['weakness_analysis']
            ),
            'strength_name': chains['strength_name'],
            'weakness_name': chains['weakness_name']
        })
        | PromptTemplate.from_template(
            """
            Translate and analyze the provided company's strengths ({strength_name}) and weaknesses ({weakness_name}):

            {info}

            - Analyze the company's strengths and weaknesses based on the provided data, focusing on key business management insights.
            - Provide specific and actionable recommendations, grounded in the analysis, to capitalize on strengths and address weaknesses.
            - Highlight critical trends and figures as evidence to support your recommendations.
            - Ensure that the recommendations are concise (limited to 5 lines) and translated into Korean in a professional tone.

            # Output Format
            - The response should be in Korean.
            - Limit the response to 5 lines or less, with a clear and cohesive structure.
            - Write in a continuous paragraph format that flows smoothly between data points, avoiding lists or bullet points.

            # Guidelines
            1. Base your response objectively on the given data without speculation.
            2. Use specific figures and trends to support your analysis.
            3. Provide practical recommendations that are directly actionable.
            4. Maintain clarity and professionalism in both the analysis and the translation to Korean.
            """
        ) 
        | insight_llm 
        | StrOutputParser()
    )

    return final_chain

# 실행

# 단일 회사 정보만을 필터링하여 사용 (예: 첫 번째 회사)
company_list = df_company_info['기업명'].unique()[:3]
for firm in company_list:
    firm_data = df_company_info[df_company_info['기업명'] == firm]
    final_chain = create_final_chain(firm_data)
    print("\n", '#'*10, f'Final Comment for {firm}', '#'*10)
    result = final_chain.invoke(firm_data)
    print('\n')


In [26]:
pprint(final_chain)

{
  strength_result: RunnableLambda(itemgetter('strength')),
  weakness_result: RunnableLambda(itemgetter('weakness'))
}
| RunnableLambda(...)
| PromptTemplate(input_variables=['info'], template="\n        Translate and analyze the provided company's strengths and weaknesses:\n\n        {info}\n\n        - Analyze the company's strengths and weaknesses based on the provided data, focusing on key business management insights.\n        - Provide specific and actionable recommendations, grounded in the analysis, to capitalize on strengths and address weaknesses.\n        - Highlight critical trends and figures as evidence to support your recommendations.\n        - Ensure that the recommendations are concise (limited to 5 lines) and translated into Korean in a professional tone.\n\n        # Output Format\n        - The response should be in Korean, using natural and connected language.\n        - Limit the response to 5 lines or less, with a clear and cohesive structure.\n        - Write